In this file, we:

- Create a mapping list to match german with english country names
- Create country groups
- Calculate measures for the country groups
- Search for important features in quality of goverment data

**Result: agg_data4.csv**

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# Load data
agg_data = pd.read_csv('agg_data3.csv')
qog_data = pd.read_csv('qog_std_ts_jan24.csv', low_memory=False)

We replace all missing value with zero.

In [2]:
qog_data = qog_data.fillna(0)

Next, we create country and country group mappings and calculate mean variable values from qog_data for the country groups.

In [3]:
# Country mapping dictionary
country_mapping = {
    'Ägypten': 'Egypt',
    'Äthiopien': 'Ethiopia',
    'Afrika (regional)': 'Afrika (regional)',
    'Albanien': 'Albania',
    'Alle Entwicklungsländer': 'Alle Entwicklungsländer',
    'Armenien': 'Armenia',
    'Aserbaidschan': 'Azerbaijan',
    'Zentralamerika (BCIE)': 'Zentralamerika (BCIE)',
    'Bangladesch': 'Bangladesh',
    'Benin': 'Benin',
    'Bolivien': 'Bolivia (Plurinational State of)',
    'Bosnien und Herzegowina': 'Bosnia and Herzegovina',
    'Brasilien': 'Brazil',
    'Bulgarien': 'Bulgaria',
    'Burkina Faso': 'Burkina Faso',
    'Burundi': 'Burundi',
    'Karibik (CARICOM)': 'Karibik (CARICOM)',
    'Afrika (CEMAC)': 'Central African Republic (the)',
    'Chile': 'Chile',
    'Volksrepublik China': 'China',
    'Costa Rica': 'Costa Rica',
    'Elfenbeinküste': 'Côte d\'Ivoire',
    'Dominikanische Republik': 'Dominican Republic (the)',
    'Dschibuti': 'Djibouti',
    'Afrika (ECOWAS)': 'Afrika (ECOWAS)',
    'Ecuador': 'Ecuador',
    'El Salvador': 'El Salvador',
    'Mosambik': 'Mozambique',
    'Eritrea': 'Eritrea',
    'Georgien': 'Georgia',
    'Ghana': 'Ghana',
    'Guatemala': 'Guatemala',
    'Guinea': 'Guinea',
    'Guyana': 'Guyana',
    'Honduras': 'Honduras',
    'Peru': 'Peru',
    'Belarus': 'Belarus',
    'Lateinamerika (regional)': 'Lateinamerika (regional)',
    'Indien': 'India',
    'Indonesien': 'Indonesia',
    'Kaukasus (regional)': 'Kaukasus (regional)',
    'Kroatien': 'Croatia',
    'Papua-Neuguinea': 'Papua New Guinea',
    'Russland': 'Russian Federation (the)',
    'Ukraine': 'Ukraine',
    'Philippinen': 'Philippines (the)',
    'Irak': 'Iraq',
    'Israel': 'Israel',
    'Jemen': 'Yemen',
    'Jordanien': 'Jordan',
    'Naher und Mittlerer Osten': 'Naher und Mittlerer Osten',
    'Kambodscha': 'Cambodia',
    'Laos': 'Lao People\'s Democratic Republic (the)',
    'Kamerun': 'Cameroon',
    'Kap Verde (Cabo Verde)': 'Cabo Verde',
    'Kenia': 'Kenya',
    'Kirgisistan': 'Kyrgyzstan',
    'Kolumbien': 'Colombia',
    'Demokratische Republik Kongo': 'Congo (the Democratic Republic of the)',
    'Kosovo': 'Kosovo',
    'Südostasien (M.R.C.)': 'Südostasien (M.R.C.)',
    'Lesotho': 'Lesotho',
    'Liberia': 'Liberia',
    'Madagaskar': 'Madagascar',
    'Malawi': 'Malawi',
    'Mali': 'Mali',
    'Marokko': 'Morocco',
    'Mauretanien': 'Mauritania',
    'Mauritius': 'Mauritius',
    'Nordmazedonien': 'North Macedonia',
    'Mexiko': 'Mexico',
    'Moldau': 'Moldova (the Republic of)',
    'Mongolei': 'Mongolia',
    'Montenegro': 'Montenegro',
    'Namibia': 'Namibia',
    'Nepal': 'Nepal',
    'Nicaragua': 'Nicaragua',
    'Niger': 'Niger (the)',
    'Nigeria': 'Nigeria',
    'Pakistan': 'Pakistan',
    'Palästina': 'Palestine',
    'Paraguay': 'Paraguay',
    'Ruanda': 'Rwanda',
    'Rumänien': 'Romania',
    'Südosteuropa': 'Südosteuropa',
    'Sambia': 'Zambia',
    'Senegal': 'Senegal',
    'Serbien': 'Serbia',
    'Simbabwe': 'Zimbabwe',
    'Somalia': 'Somalia',
    'Sri Lanka': 'Sri Lanka',
    'Südafrika': 'South Africa',
    'Mittel- und Südosteuropa (regional)': 'Mittel- und Südosteuropa (regional)',
    'Tadschikistan': 'Tajikistan',
    'Tansania': 'Tanzania, the United Republic of',
    'Osttimor': 'Timor-Leste',
    'Togo': 'Togo',
    'Tschad': 'Chad',
    'Tunesien': 'Tunisia',
    'Türkei': 'Turkey',
    'Uganda': 'Uganda',
    'Usbekistan': 'Uzbekistan',
    'Vietnam': 'Viet Nam',
    'Zentralafrikanische Republik': 'Central African Republic (the)',
    'Zentralamerika (CTPT)': 'Zentralamerika (CTPT)',
    'Kasachstan': 'Kazakhstan',
    'Mittlerer Osten und Nordafrika': 'Mittlerer Osten und Nordafrika',
    'Sierra Leone': 'Sierra Leone',
    'Syrien': 'Syrian Arab Republic (the)',
}

# Dictionary with country groups and corresponding countries
country_groups = {
    'Afrika (regional)': ['Egypt', 'Ethiopia', 'Central African Republic (the)', 'Ghana', 'Kenya', 'Nigeria', 'South Africa', 'Tanzania, the United Republic of', 'Uganda', 'Zambia', 'Zimbabwe'],
    'Zentralamerika (BCIE)': ['Costa Rica', 'El Salvador', 'Guatemala', 'Honduras', 'Nicaragua'],
    'Karibik (CARICOM)': ['Guyana'],
    'Afrika (ECOWAS)': ['Benin', 'Burkina Faso', 'Côte d\'Ivoire', 'Ghana', 'Guinea', 'Liberia', 'Nigeria', 'Senegal', 'Sierra Leone', 'Togo'],
    'Kaukasus (regional)': ['Armenia', 'Azerbaijan', 'Georgia'],
    'Südostasien (M.R.C.)': ['Cambodia', 'Lao People\'s Democratic Republic (the)', 'Vietnam'],
    'Lateinamerika (regional)': ['Bolivia (Plurinational State of)', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Peru'],
    'Naher und Mittlerer Osten': ['Israel', 'Jordan', 'Syrian Arab Republic (the)', 'Yemen'],
    'Mittel- und Südosteuropa (regional)': ['Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Montenegro', 'North Macedonia', 'Romania', 'Serbia'],
    'Zentralamerika (CTPT)': ['Costa Rica', 'El Salvador', 'Guatemala', 'Honduras', 'Nicaragua'],
    'Mittlerer Osten und Nordafrika': ['Israel', 'Jordan', 'Syrian Arab Republic (the)', 'Yemen']
}

# Calculating mean values for country groups
grouped_data = []

for group, countries in country_groups.items():
    for year in qog_data['year'].unique():
        group_data = qog_data[qog_data['cname'].isin(countries) & (qog_data['year'] == year)]
        if not group_data.empty:
            mean_values = group_data.mean(numeric_only=True)
            mean_values['cname'] = group
            mean_values['year'] = year
            grouped_data.append(mean_values)

grouped_df = pd.DataFrame(grouped_data)
qog_data = pd.concat([qog_data, grouped_df], ignore_index=True)

From agg_data we select the columns country, evaluation year, overall rating. We select all numerical columns from qog_data and merge the two dataframes on country and evaluation year. By doing so, we make the simplifying assumption that the census data does not change during the project phase. We use only the data from the respective evaluation year of the project.

In [4]:
# Convert countries in agg_data to English
agg_data['country'] = agg_data['country'].map(country_mapping)

agg_data = agg_data[['country', 'eval_year', 'overall_rating']]
agg_data.rename(columns={'country': 'cname', 'eval_year': 'year'}, inplace=True)

# Select only numeric columns in qog_data, except 'year'
numerical_cols = qog_data.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols = [col for col in numerical_cols if col != 'year']

# Merge agg_data with qog_data on 'cname' and 'year'
merged_data = pd.merge(agg_data, qog_data[['cname', 'year'] + numerical_cols], how='left', on=['cname', 'year'])

# Remove rows with missing values
merged_data = merged_data.dropna()

Here, we separate the complete dataframe merged_data (agg_data with more variables) in to train_test and validation dataframe. We only use the train_test dataframe for feature selection. The validation dataframe is used for final model valuation.

In [5]:
# Split data into training and validation datasets
validation_data = merged_data.sample(frac=0.1, random_state=123)
train_test_data = merged_data.drop(validation_data.index)

# Specifying X and y. Standardizing X
X = train_test_data.drop(columns=['overall_rating', 'cname', 'year'])
y = train_test_data['overall_rating']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Next, we perform feature selection using a random forest and print out the 5 most relevant features.

In [6]:
# Random Forest Feature Selection
rf = RandomForestRegressor(random_state=123)
rf.fit(X_scaled, y)

importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
top_secondary_features = X.columns[indices[:5]].tolist()
print("The top secondary features are:", top_secondary_features)

The top secondary features are: ['wbgi_gee', 'wdi_expedus', 'wdi_gdpind', 'gpi_ss', 'bci_bcistd']


Finally, we add the 5 selected variables to agg_data and export it to a CSV file.

In [7]:
agg_data3 = pd.read_csv('agg_data3.csv')
agg_data3['country'] = agg_data3['country'].map(country_mapping)
merged_data_full = pd.merge(agg_data3, qog_data[['cname', 'year'] + list(top_secondary_features)], how='left', left_on=['country', 'eval_year'], right_on=['cname', 'year'])

for feature in top_secondary_features:
    agg_data3[feature] = merged_data_full[feature]

# Replace NaN values with 0 in the new columns
agg_data3[top_secondary_features] = agg_data3[top_secondary_features].fillna(0)

# Save the updated file
agg_data3.to_csv('agg_data4.csv', index=False)

## Explanation of Selected Variables

* wbgi_gee (Government Effectiveness - Estimate):

Description: This variable combines responses on the quality of public service provision, the quality of the bureaucracy, the competence of civil servants, the independence of the civil service from political pressures, and the credibility of the government’s commitment to policies.
* wdi_expedus (Expenditure on Secondary Education):

Description: This variable represents the expenditure on secondary education as a percentage of total general government expenditure on education. It is calculated by dividing government expenditure on secondary education by total government expenditure on education and multiplying by 10034.
* wdi_gdpind (Industry, Value Added as % of GDP):

Description: This variable includes value added in mining, manufacturing, construction, electricity, water, and gas. It represents the net output of a sector after adding up all outputs and subtracting intermediate inputs, without making deductions for depreciation of fabricated assets or depletion and degradation of natural resources.
* gpi_ss (Societal Safety and Security):

Description: This variable is one of the three subdomains of the Global Peace Index. It measures societal safety and security, scaled from 1 to 5, with 5 being less secure. It includes indicators such as low crime rates, minimal terrorist activity, harmonious relations with neighboring countries, a stable political scene, and a small proportion of the population being internally displaced or made refugees.
* bci_bcistd (Standard Deviation of the Bayesian Corruption Indicator):

Description: This variable represents the standard deviation of the Bayesian Corruption Index, which is a composite index of the perceived overall level of corruption.